In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras
import cv2
import matplotlib.pyplot as plt 
import math

In [2]:
#Data Pre-processing Steps
#Creating lists of the images
base_dir='C:/Users/playf/OneDrive/Documents/UBC/Thesis/Custom_Model_Code/data/labels/'
train_paths=np.loadtxt(base_dir+'keypoints_spaces_train.txt',usecols=0,dtype=str)
valid_paths=np.loadtxt(base_dir+'keypoints_spaces_valid.txt',usecols=0,dtype=str)
test_paths=np.loadtxt(base_dir+'keypoints_spaces_test.txt',usecols=0,dtype=str)

train_images=[]
valid_images=[]
test_images=[]
for path in train_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    train_images.append(img_norm)

for path in valid_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    valid_images.append(img_norm)

for path in test_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    test_images.append(img_norm)


#Converting image lists into a (num_images,512,512,1) np.array
train_images=np.array(train_images,dtype=np.float32)
num_train=np.shape(train_images)[0]
train_images=np.expand_dims(train_images,axis=-1)
train_images=train_images.reshape(num_train,512,512,1)
print(np.shape(train_images))

test_images=np.array(test_images,dtype=np.float32)
num_test=np.shape(test_images)[0]
test_images=np.expand_dims(test_images,axis=-1)
test_images=test_images.reshape(num_test,512,512,1)
print(np.shape(test_images))

valid_images=np.array(valid_images,dtype=np.float32)
num_valid=np.shape(valid_images)[0]
valid_images=np.expand_dims(valid_images,axis=-1)
valid_images=valid_images.reshape(num_valid,512,512,1)
print(np.shape(valid_images))



(647, 512, 512, 1)
(82, 512, 512, 1)
(82, 512, 512, 1)


In [3]:
#Creating np array of the labels
train_keypoints=np.loadtxt(base_dir+'keypoints_spaces_train.txt',usecols=[1,2,3,4,5,6,7,8])
valid_keypoints=np.loadtxt(base_dir+'keypoints_spaces_valid.txt',usecols=[1,2,3,4,5,6,7,8])
test_keypoints=np.loadtxt(base_dir+'keypoints_spaces_test.txt',usecols=[1,2,3,4,5,6,7,8])

In [4]:
#Define our CNN model
def build_eyecorner_detection_model(input_shape):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the feature maps
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8, activation='linear'))  # Output layer with 8 values (4 keypoints (x and y for both eyes' corners))

    return model

In [6]:
#Define input shape and initialize the model
input_shape=(512,512,1) #Input image is a 512x512 grayscale image
model=build_eyecorner_detection_model(input_shape)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 253, 253, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 64)      0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1016064)           0         
                                                                 
 dense (Dense)               (None, 128)               1

In [7]:
#Training settings
epochs=220
batch_size=32

In [8]:
#Creating epoch logger
checkpoint_path = "C:/Users/playf/OneDrive/Documents/UBC/Thesis/Custom_Model_Code/models/cp-{epoch:04d}.ckpt"
n_batches = np.shape(train_images)[0] / batch_size
n_batches = math.ceil(n_batches)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)
#Save the initial epoch
model.save_weights(checkpoint_path.format(epoch=0))

In [9]:
#Training:
history=model.fit(train_images,
                  train_keypoints,
                  validation_data=(valid_images,valid_keypoints),
                  batch_size=batch_size,
                  epochs=epochs,
                  callbacks=[cp_callback])

Epoch 1/220
21/21 [==============================] - 130s 6s/step - loss: 9.5115 - mae: 1.1658 - val_loss: 0.1564 - val_mae: 0.3322
Epoch 2/220
21/21 [==============================] - 103s 5s/step - loss: 0.1526 - mae: 0.3200 - val_loss: 0.0840 - val_mae: 0.2306
Epoch 3/220
21/21 [==============================] - 99s 5s/step - loss: 0.1079 - mae: 0.2628 - val_loss: 0.0416 - val_mae: 0.1511
Epoch 4/220
21/21 [==============================] - 102s 5s/step - loss: 0.0790 - mae: 0.2229 - val_loss: 0.0313 - val_mae: 0.1336
Epoch 5/220
20/21 [===========================>..] - ETA: 5s - loss: 0.0611 - mae: 0.1934 
Epoch 5: saving model to C:/Users/playf/OneDrive/Documents/UBC/Thesis/Custom_Model_Code/models\cp-0005.ckpt
21/21 [==============================] - 110s 5s/step - loss: 0.0610 - mae: 0.1930 - val_loss: 0.0201 - val_mae: 0.1022
Epoch 6/220
21/21 [==============================] - 111s 5s/step - loss: 0.0540 - mae: 0.1788 - val_loss: 0.0179 - val_mae: 0.0962
Epoch 7/220
21/21 [===

UnknownError: {{function_node __wrapped__SaveV2_dtypes_27_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to WriteFile: \\?\C:\Users\playf\OneDrive\Documents\UBC\Thesis\Custom_Model_Code\models\cp-0160.ckpt_temp\part-00000-of-00001.data-00000-of-00001.tempstate17407986665170637874 : There is not enough space on the disk.
; operation in progress [Op:SaveV2]

In [5]:
input_shape=(512,512,1)
#Retrain model with latest epoch
latest=tf.train.latest_checkpoint("E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/NewTrain")
#Create new model instance
model=build_eyecorner_detection_model(input_shape)
model.load_weights(latest)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
epochs=140
batch_size=32

#Create new epoch loader
#Creating epoch logger
checkpoint_path = "E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/NewTrain/cp-{epoch:04d}.ckpt"
n_batches = np.shape(train_images)[0] / batch_size
n_batches = math.ceil(n_batches)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=15*n_batches) #Save every 15 epochs
#Save the initial epoch
model.save_weights(checkpoint_path.format(epoch=0))
history=model.fit(train_images,
                  train_keypoints,
                  validation_data=(valid_images,valid_keypoints),
                  batch_size=batch_size,
                  epochs=epochs,
                  callbacks=[cp_callback])

#Save the entire (most recent epoch) model:
model.save('E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/NewTrain/eyecorner_Aug2.keras')
model.save('E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/NewTrain/models/eyecorner_Aug2.h5')



Epoch 1/140
21/21 [==============================] - 137s 5s/step - loss: 0.0027 - mae: 0.0336 - val_loss: 0.0036 - val_mae: 0.0268
Epoch 2/140
21/21 [==============================] - 97s 5s/step - loss: 0.0027 - mae: 0.0334 - val_loss: 0.0037 - val_mae: 0.0276
Epoch 3/140
21/21 [==============================] - 97s 5s/step - loss: 0.0025 - mae: 0.0329 - val_loss: 0.0038 - val_mae: 0.0273
Epoch 4/140
21/21 [==============================] - 97s 5s/step - loss: 0.0025 - mae: 0.0327 - val_loss: 0.0035 - val_mae: 0.0266
Epoch 5/140
21/21 [==============================] - 104s 5s/step - loss: 0.0027 - mae: 0.0326 - val_loss: 0.0036 - val_mae: 0.0257
Epoch 6/140
21/21 [==============================] - 112s 5s/step - loss: 0.0025 - mae: 0.0330 - val_loss: 0.0035 - val_mae: 0.0270
Epoch 7/140
21/21 [==============================] - 119s 6s/step - loss: 0.0026 - mae: 0.0329 - val_loss: 0.0036 - val_mae: 0.0272
Epoch 8/140
21/21 [==============================] - 119s 6s/step - loss: 0.002

c:\Users\playf\OneDrive\Documents\UBC\Thesis\Custom_Model_Code\custom_model_venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
#Save the entire (most recent epoch) model:
save_path='E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/FullModel/'
tf.saved_model.save(model,save_path)

INFO:tensorflow:Assets written to: E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/FullModel/assets


INFO:tensorflow:Assets written to: E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/FullModel/assets


In [10]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(['Training', 'Validation'])
plt.show()

NameError: name 'history' is not defined

In [7]:
loss, acc = model.evaluate(test_images,test_keypoints)

print('Loss:', loss)
print('Accuracy:', 100*acc)

3/3 [==============================] - 4s 1s/step - loss: 8.9361e-04 - mae: 0.0201
Loss: 0.0008936140220612288
Accuracy: 2.0114488899707794
